In [1]:
import geopandas as gpd
from simpledbf import Dbf5
import time
from glob import glob

PyTables is not installed. No support for HDF output.


In [2]:
fi = r"D:\projects\RD\night_fire\modis_fire_points\DL_FIRE_M6_50643\fire_archive_M6_50643.dbf"
shp_fi = r"D:\projects\RD\night_fire\modis_fire_points\DL_FIRE_M6_50643\fire_archive_M6_50643.shp"

In [3]:
fire_pts_folders = !ls -d ../modis_fire_points/*
dbf_files = [glob(folder + '/*.dbf')[0] for folder in fire_pts_folders]

years, type0, type1, type2, type3 = [], [], [], [], []
for fi in [dbf_files[-1]]:
    dbf = Dbf5(fi)
    df_dbf = dbf.to_dataframe()
    
    types = df_dbf.groupby('TYPE')
    try:
        year = int(df_dbf.ACQ_DATE[0].split('-')[0])
    except:
        year = df_dbf.ACQ_DATE[0].year
        
    years.append(year)
    type0.append(list(types)[0][1].shape[0])
    type1.append(list(types)[1][1].shape[0])
    type2.append(list(types)[2][1].shape[0])
    type3.append(list(types)[3][1].shape[0])
    
    print('on year {}'.format(year))
    for name, grp in types:
        print('TYPE: {}, size: {}    {:.2f}'.format(name, grp.shape[0], grp.shape[0]/df_dbf.shape[0]))
    
    
    df_type0 = df_dbf.query('TYPE == 0')
    for conf in [0,10,20,30]:
        df_sub = df_type0.query('CONFIDENCE > {}'.format(conf))
        print('confidence > {}, using {:.2f}% of data'.format(conf, (df_sub.shape[0]/df_type0.shape[0])*100))
    print('******************************')

on year 2011
TYPE: 0, size: 4575140    0.97
TYPE: 1, size: 1333    0.00
TYPE: 2, size: 124053    0.03
TYPE: 3, size: 22579    0.00
confidence > 0, using 99.36% of data
confidence > 10, using 99.16% of data
confidence > 20, using 98.05% of data
confidence > 30, using 95.05% of data
******************************


In [ ]:
counts_df = pd.DataFrame({'year': years, 'type0':type0, 'type1':type1, 'type2':type2, 'type3':type3})